In [ ]:
from datetime import date
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine

In [2]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

In [3]:
# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

In [10]:
# Consulta SQL para obtener los servicios y sus estados
query = """
SELECT 
    s.id AS servicio_id,
    os.cliente_id,
    sede.sede_id,
    COALESCE(s.mensajero3_id, COALESCE(s.mensajero2_id, s.mensajero_id)) AS mensajero_final_id,
    es.fecha AS fecha_estado,
    es.hora AS hora_estado
FROM mensajeria_servicio s
JOIN mensajeria_estadosservicio es 
    ON s.id = es.servicio_id
JOIN mensajeria_origenservicio os 
    ON s.origen_id = os.id
JOIN sede 
    ON sede.cliente_id = os.cliente_id 
    AND sede.ciudad_id = os.ciudad_id
ORDER BY s.id, es.fecha, es.hora


"""

In [14]:
# Leer datos y dimensiones
df = pd.read_sql(query, fuente_conn)
dim_fecha = pd.read_sql_table('dim_fecha', bodega_conn)
dim_cliente = pd.read_sql_table('dim_cliente', bodega_conn)
dim_mensajero = pd.read_sql_table('dim_mensajero', bodega_conn)
dim_sede = pd.read_sql_table('dim_sede', bodega_conn)

ValueError: Table dim_fecha not found

In [ ]:
# Convertir fecha_estado a datetime
df['fecha_estado'] = pd.to_datetime(df['fecha_estado']).dt.date
dim_fecha['fecha'] = pd.to_datetime(dim_fecha['fecha']).dt.date

In [ ]:
# Calcular tiempo de entrega por servicio
def calcular_tiempo_entrega(grupo):
    estados_ordenados = grupo.sort_values(['fecha_estado', 'hora_estado'])
    if len(estados_ordenados) >= 2:
        inicio = pd.to_datetime(f"{estados_ordenados.iloc[0]['fecha_estado']} {estados_ordenados.iloc[0]['hora_estado']}")
        fin = pd.to_datetime(f"{estados_ordenados.iloc[-1]['fecha_estado']} {estados_ordenados.iloc[-1]['hora_estado']}")
        return (fin - inicio).total_seconds() / 3600  # Convertir a horas
    return None

df_tiempo_entrega = df.groupby('servicio_id').apply(calcular_tiempo_entrega).reset_index(name='tiempo_entrega')


In [ ]:
# Obtener solo el primer estado de cada servicio para la fecha
df_servicios = df.sort_values(['fecha_estado', 'hora_estado']).groupby('servicio_id').first().reset_index()


In [ ]:
# Combinar con los tiempos de entrega
df_servicios = df_servicios.merge(df_tiempo_entrega, on='servicio_id', how='left')


In [ ]:
# Realizar los merges con las dimensiones
hecho_diario = df_servicios.merge(
    dim_fecha[['key_dim_fecha', 'fecha']], 
    left_on='fecha_estado', 
    right_on='fecha',
    how='left'
)

hecho_diario = hecho_diario.merge(
    dim_cliente[['key_dim_cliente', 'cliente_id']], 
    on='cliente_id',
    how='left'
)

hecho_diario = hecho_diario.merge(
    dim_mensajero[['key_dim_mensajero', 'mensajero_id']], 
    left_on='mensajero_final_id',
    right_on='mensajero_id',
    how='left'
)

hecho_diario = hecho_diario.merge(
    dim_sede[['key_dim_sede', 'sede_id']],
    on='sede_id',
    how='left'
)

In [ ]:
# Seleccionar columnas finales
columnas_finales = [
    'servicio_id',
    'key_dim_fecha',
    'key_dim_cliente',
    'key_dim_mensajero',
    'key_dim_sede',
    'tiempo_entrega'
]

hecho_diario = hecho_diario[columnas_finales]

In [ ]:
# Agregar fecha de carga
hecho_diario['saved'] = date.today()


In [ ]:
# Verificaciones
print("\nInformación del DataFrame:")
print(hecho_diario.info())

print("\nEstadísticas de tiempo de entrega:")
print(hecho_diario['tiempo_entrega'].describe())

print("\nVerificar valores nulos:")
print(hecho_diario.isnull().sum())

In [ ]:
# Guardar en la bodega
hecho_diario.to_sql(
    'hecho_entrega_servicio_diaria', 
    bodega_conn, 
    if_exists='replace', 
    index=False
)

In [ ]:
#Pregunta 1: ¿En qué meses del año los clientes solicitan más servicios de mensajería?

'''
SELECT 
    df.mes,
    COUNT(h.servicio_id) AS total_servicios
FROM hecho_entrega_servicio_dia h
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
GROUP BY df.mes
ORDER BY total_servicios DESC;
'''

In [ ]:
#Pregunta 2: ¿Cuáles son los días donde más solicitudes hay?

'''
SELECT 
    df.dia_semana,
    COUNT(h.servicio_id) AS total_servicios
FROM hecho_entrega_servicio_dia h
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
GROUP BY df.dia_semana
ORDER BY total_servicios DESC;
'''

In [ ]:
#Pregunta 3: Número de servicios solicitados por cliente y por mes

'''
SELECT 
    dc.nombre AS nombre_cliente,
    df.mes,
    COUNT(h.servicio_id) AS total_servicios
FROM hecho_entrega_servicio_dia h
JOIN dim_cliente dc ON h.key_dim_cliente = dc.key_dim_cliente
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
GROUP BY dc.nombre, df.mes
ORDER BY dc.nombre, df.mes;
'''